In [42]:
#using MLDatasets
using ImageCore
using Flux, Flux.Data.FashionMNIST, Images, Statistics
using Flux: onehotbatch, onecold, crossentropy, throttle
using Base.Iterators: repeated, partition
using Printf, BSON

┌ Info: Precompiling BSON [fbb218c0-5317-5bc6-957e-2ee96dd4b1f0]
└ @ Base loading.jl:1192


In [29]:
images = Flux.Data.FashionMNIST.images();
labels = Flux.Data.FashionMNIST.labels();

┌ Info: Downloading Fashion-MNIST dataset
└ @ Flux.Data.FashionMNIST /home/emory/.juliapro/JuliaPro_v1.0.5-2/packages/Flux/NpkMm/src/data/fashion-mnist.jl:16
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed


 24 25.1M   24 6255k    0     0  25105      0  0:

 30 25.1M   30 7913k    0     0  21038     

 36 25.1M   36 9509k    0     0  18905     

 42 25.1M   42 10.7M    0     0  17532      0

...

 84 25.1M   84 21.2M    0     0  17199      0

 93 25.1M   93 23.4M    0    

100 25.1M  100 25.1M    0     0  17685      0  0:24:54  0:24:54 --:--:-- 13581
┌ Info: Downloading Fashion-MNIST dataset
└ @ Flux.Data.FashionMNIST /home/emory/.juliapro/JuliaPro_v1.0.5-2/packages/Flux/NpkMm/src/data/fashion-mnist.jl:16
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 29515  100 29515    0     0  19929      0  0:00:01  0:00:01 --:--:-- 19929
┌ Info: Downloading Fashion-MNIST dataset
└ @ Flux.Data.FashionMNIST /home/emory/.juliapro/JuliaPro_v1.0.5-2/packages/Flux/NpkMm/src/data/fashion-mnist.jl:16
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed


 72 4318k   72 3139k    0     0  18344     

100 4318k  100 4318k    0     0  18460      0  0:03:59  0:03:59 --:--:-- 26622
┌ Info: Downloading Fashion-MNIST dataset
└ @ Flux.Data.FashionMNIST /home/emory/.juliapro/JuliaPro_v1.0.5-2/packages/Flux/NpkMm/src/data/fashion-mnist.jl:16
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5148  100  5148    0     0   7061      0 --:--:-- --:--:-- --:--:--  7052


In [45]:
function make_minibatch(X, Y, idxs)
    X_batch = Array{Float32}(undef, size(X[1])..., 1, length(idxs))
    for i in 1:length(idxs)
        X_batch[:, :, :, i] = Float32.(X[idxs[i]])
    end
    Y_batch = onehotbatch(Y[idxs], 0:9)
    return (X_batch, Y_batch)
end
batch_size = 128
mb_idxs = partition(1:length(images), batch_size)
train_set = [make_minibatch(images, labels, i) for i in mb_idxs]

469-element Array{Tuple{Array{Float32,4},Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}},1}:
 ([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

...

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [false true … false false; false false … false false; … ; false false … false false; true false … false false])                              
 ([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

...

[0.0 0.0 … 0.0

In [49]:
model = Chain(
    # First convolution, operating upon a 28x28 image
    Conv((3, 3), 1=>16, pad=(1,1), relu),
    x -> maxpool(x, (2,2)),

    # Second convolution, operating upon a 14x14 image
    Conv((3, 3), 16=>32, pad=(1,1), relu),
    x -> maxpool(x, (2,2)),

    # Third convolution, operating upon a 7x7 image
    Conv((3, 3), 32=>32, pad=(1,1), relu),
    x -> maxpool(x, (2,2)),

    # Reshape 3d tensor into a 2d one, at this point it should be (3, 3, 32, N)
    # which is where we get the 288 in the `Dense` layer below:
    x -> reshape(x, :, size(x, 4)),
    Dense(288, 10),

    # Finally, softmax to get nice probabilities
    softmax,
);

In [51]:
# Prepare test set as one giant minibatch:
test_imgs = Flux.Data.FashionMNIST.images(:test)
test_labels = Flux.Data.FashionMNIST.labels(:test)
test_set = make_minibatch(test_imgs, test_labels, 1:length(test_imgs))

(Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

...

Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], Bool[false false … false false; false false … true false; … ; false false … false false; true false … false false])

In [58]:
# `loss()` calculates the crossentropy loss between our prediction `y_hat`
# (calculated from `model(x)`) and the ground truth `y`.  We augment the data
# a bit, adding gaussian random noise to our image to make it more robust.
function L(x, y)
    # We augment `x` a little bit here, adding in random noise
    x_aug = x .+ 0.1f0*randn(eltype(x), size(x))

    y_hat = model(x_aug)
    return crossentropy(y_hat, y)
end
accuracy(x, y) = mean(onecold(model(x)) .== onecold(y))

# Train our model with the given training set using the ADAM optimizer and
# printing out performance against the test set as we go.
opt = ADAM(0.001)

ADAM(0.001, (0.9, 0.999), IdDict{Any,Any}())

In [61]:
model(train_set[1][1])

10×128 Array{Float32,2}:
 0.102445   0.102955   0.0986288  …  0.0977012  0.0991384  0.0966357
 0.0966567  0.097253   0.102839      0.0920132  0.101476   0.102331 
 0.0988827  0.0976921  0.105376      0.100529   0.0997557  0.105922 
 0.0611958  0.0617126  0.0798424     0.0594599  0.0672739  0.0668106
 0.103724   0.102868   0.102632      0.099457   0.106876   0.10688  
 0.0971879  0.0967925  0.0970954  …  0.102142   0.0972546  0.0975481
 0.0852088  0.0904074  0.0942056     0.0979503  0.0880971  0.0932243
 0.127485   0.111506   0.105185      0.120672   0.11046    0.10747  
 0.119949   0.126535   0.109859      0.122219   0.119853   0.118681 
 0.107265   0.112278   0.104336      0.107857   0.109816   0.104498 

In [62]:
@info("Beginning training loop...")
best_acc = 0.0
last_improvement = 0
for epoch_idx in 1:100
    global best_acc, last_improvement
    # Train for a single epoch
    Flux.train!(L, params(model), train_set, opt)

    # Calculate accuracy:
    acc = accuracy(test_set...)
    @info(@sprintf("[%d]: Test accuracy: %.4f", epoch_idx, acc))
    
    # If our accuracy is good enough, quit out.
    if acc >= 0.999
        @info(" -> Early-exiting: We reached our target accuracy of 99.9%")
        break
    end

    # If this is the best accuracy we've seen so far, save the model out
    if acc >= best_acc
        @info(" -> New best accuracy! Saving model out to mnist_conv.bson")
        BSON.@save "mnist_conv.bson" model epoch_idx acc
        best_acc = acc
        last_improvement = epoch_idx
    end

    # If we haven't seen improvement in 5 epochs, drop our learning rate:
    if epoch_idx - last_improvement >= 5 && opt.eta > 1e-6
        opt.eta /= 10.0
        @warn(" -> Haven't improved in a while, dropping learning rate to $(opt.eta)!")

        # After dropping learning rate, give it a few epochs to improve
        last_improvement = epoch_idx
    end

    if epoch_idx - last_improvement >= 10
        @warn(" -> We're calling this converged.")
        break
    end
end

┌ Info: Beginning training loop...
└ @ Main In[62]:1
┌ Info: [1]: Test accuracy: 0.8247
└ @ Main In[62]:11
┌ Info:  -> New best accuracy! Saving model out to mnist_conv.bson
└ @ Main In[62]:21
┌ Info: [2]: Test accuracy: 0.8536
└ @ Main In[62]:11
┌ Info:  -> New best accuracy! Saving model out to mnist_conv.bson
└ @ Main In[62]:21
┌ Info: [3]: Test accuracy: 0.8657
└ @ Main In[62]:11
┌ Info:  -> New best accuracy! Saving model out to mnist_conv.bson
└ @ Main In[62]:21
┌ Info: [4]: Test accuracy: 0.8727
└ @ Main In[62]:11
┌ Info:  -> New best accuracy! Saving model out to mnist_conv.bson
└ @ Main In[62]:21
┌ Info: [5]: Test accuracy: 0.8737
└ @ Main In[62]:11
┌ Info:  -> New best accuracy! Saving model out to mnist_conv.bson
└ @ Main In[62]:21
┌ Info: [6]: Test accuracy: 0.8788
└ @ Main In[62]:11
┌ Info:  -> New best accuracy! Saving model out to mnist_conv.bson
└ @ Main In[62]:21
┌ Info: [7]: Test accuracy: 0.8797
└ @ Main In[62]:11
┌ Info:  -> New best accuracy! Saving model out to mn

### This notebook is based on
https://spcman.github.io/getting-to-know-julia/deep-learning/vision/flux-cnn-zoo/
https://colab.research.google.com/github/GdMacmillan/ml_flux_tutorial/blob/master/ML_Flux_Tutorial.ipynb#scrollTo=74SWEXwTH0yM